Соберем это все в свёрточную сеть

Теперь мы готовы собрать нашу первую свеёрточную сеть. 

Для примера рассмотрим архитектуру LeNet, придуманную 1998 году. Она применялась для задачи MNIST — задачи распознавания рукописных цифр.

На вход принималась чёрно-белая картинка размера 32 × 32 × 1. Сначала предлагается применить свёртку с фильтром размером 5 × 5 и обучить 6 таких фильтров, чтобы на выходе получить объем 28 × 28 × 6 (32 превратилось в 28 потому, что мы не использовали padding). Дальше предлагается использовать пулинг слой, чтобы уменьшить картинку в 2 раза, т.о. получив размеры 14 × 14 × 6.

https://lms-cdn.skillfactory.ru/assets/courseware/v1/1a51603a2c5e47854ae3a019208bae54/asset-v1:SkillFactory+MLDL+ALWAYS+type@asset+block/m3sl36.png

Количество фильтров при этом не поменяется, т.к. пулинг работает с каждым feature map независимо.

Потом применяем ещё один свёрточный слой с фильтром 5 × 5, чтобы увеличить поле обзора, и получаем размеры 10 × 10 × 16. Фильтров, которые обучаются, становится уже больше. Это делается для того, чтобы не терять информацию. 

Вновь используем пулинг слой, который сделает нашу картинку размером 5 × 5 × 16. Полученный объём предлагается вытянуть в вектор и к этому вектору применить 2 полносвязных слоя, т.е. предполагается, что в этом последнем пулинг слое в 16 признаках уже закодирована нужная информация, которая поможет нам воссоздать цифру.

После применения двух полносвязных слоёв нужен выходной слой, на котором будет 10 выходов, т.к. у нас 10 цифр, и будет применена функция softmax, которая превратит любые выходы в правильное распределение вероятностей. 

Если взять 60 000 примеров, где у нас есть картинка на входе и для неё известен класс, собрать эту архитектуру и оставить backpropagation на пару минут, то он выучит все веса на всех слоях, которые решают эту задачу практически идеально.

Как работает функция softmax?

Допустим, мы посчитали 10 выходов. Каждый из них заменим на экспоненту в степени этого выхода и пронормируем все полученные экспоненты, чтобы в сумме они давали 1. Т.к. экспонента неотрицательна, а их сумма равна 1, то получим правильное распределение вероятностей.

https://lms-cdn.skillfactory.ru/assets/courseware/v1/630df4984c912617e14901c73ad1b8f2/asset-v1:SkillFactory+MLDL+ALWAYS+type@asset+block/m3sl37.png

Кросс-энтропия

Чтобы запустить backpropagation, нужно знать какую функцию потерь использовать. Для классификации на K классов обычно используют функцию потерь, которую называют кросс-энтропия (cross-entropy).

Для каждого примера мы идем по всем возможным классам и, если пример принадлежит этому классу, то мы в потери записываем логарифм со знаком «минус» от предсказанной вероятности этого класса.

https://lms-cdn.skillfactory.ru/assets/courseware/v1/d7c1e1cd2c30e7f8b4ba23e0323db679/asset-v1:SkillFactory+MLDL+ALWAYS+type@asset+block/m3sl38.png

По графику функции -ln(х) видно, что если аргумент этой функции близок к 1, то потери близки к 0, а если аргумент близок к 0, то потери уходят в бесконечность. Это означает, что в потерях на правильном классе хочется видеть вероятность как можно более близкую к 1. Это и будет решать нашу задачу  классификации.

Если обучить нашу нейросеть и посмотреть на каждом слое изображения, на какие кусочки реагируют (имеют большую активацию) наши нейроны, то заметим, что на первом слое нейроны реагируют на простые паттерны на изображении (например, они могут находить перепады яркости под разными углами). Второй свёрточный слой уже может распознавать что-то более сложное (например, кусочек глаза) и т.д. Таким образом. нейросеть учит иерархические шаблоны.

https://lms-cdn.skillfactory.ru/assets/courseware/v1/e231859d182ac7c08b0db1f905809ca2/asset-v1:SkillFactory+MLDL+ALWAYS+type@asset+block/m3sl39.png